# Trajectory fusing and motion estimation - NGSIM

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import os

import libStep1 as s1
import libStep2 as s2
import libFileio as f
import importlib

import glob

In [5]:
importlib.reload(f)
importlib.reload(s1)
importlib.reload(s2)

# vNum = '7'
# combType = 'combined'
# srcDataDir = 'data/rawdata_' + combType + '_v' + vNum + '/'

# OR, specify input data folder
srcDataDir = 'data/v7_5_1_combined_preproc/' # NGSIM
# srcDataDir = 'data/lidar_mini_test_220326/'

print(srcDataDir)

num_cores = multiprocessing.cpu_count()

# create folder under srcDataDir for data output
try:
    os.mkdir(srcDataDir + 'output')
except FileExistsError:
    pass


data/v7_5_1_combined_preproc/


In [6]:
def veh_motion_est(data, vid):
    # get vehicle
    dataVeh = f.get_veh(data, vid)
    # est motion
    dataTemp = s1.combine_cam_motion_est_ud(dataVeh)
    return dataTemp #, np.shape(dataTemp)[0]

## i/o and step 1 and 2

In [7]:
file_list = glob.glob(srcDataDir + '*.mat')
print("Found " + str(len(file_list)) + " data files under " + srcDataDir + "\n")

for file in file_list:
    print("\n----------------------------------------------------------------------")
    print("Working on " + file + "...")

    # load all cam data
    if os.name == 'nt':
        target_file_name = file.split('\\')[-1]
    else:
        target_file_name = file.split('/')[-1]
        
    data = f.load_data(srcDataDir + target_file_name, unit='meter')

    sz = np.shape(data)

    if sz[1] < 23:
        tmp = np.zeros((sz[0], 23-sz[1]))
        print(np.shape(tmp))
        # data[:,22] = np.zeros((sz[0],))
        data = np.hstack((data, tmp))


    print(srcDataDir + target_file_name)
    # get vehicle ID list
    vidList = np.unique(data[:,0])
    print('There are ' + str(len(vidList)) + ' vehicles in this dataset.')


    ####### step 1
    startTime = time.time() # time run time
    dataStack = Parallel(n_jobs=8)(delayed(veh_motion_est)(data, vid) for vid in tqdm(vidList))
    dataX = np.concatenate(dataStack)
    print('\nruntime = {:.2f}'.format(time.time() - startTime))

    f.save_data_step1(dataX, srcDataDir + 'output/', target_file_name)

    ##################################################################
    ####### step 2
    data = dataX.copy()             
    vidList = np.unique(data[:,0])  # get vehicle ID list
    startTime = time.time()         # time run time

    ##################################################################
    ##### step 2
    dataS = data.copy()             # make a copy for stop processing
    stopInfoAll = np.zeros((1,5))   # init stop info matrix
    for vid in tqdm(vidList):
        ind = dataS[:,0]==vid
        dataVehS, isStopped, stopInfo = s2.ns_and_s_handle(dataS[ind, :])
        if isStopped:
            dataS[ind, :] = dataVehS
            stopInfoAll = np.vstack((stopInfoAll, stopInfo))
            
    stopInfoAll = stopInfoAll[1:,:]

    print('\nruntime = {:.2f}'.format(time.time() - startTime))

    f.save_data_step2(dataS, srcDataDir + 'output/', target_file_name.replace('.mat','') +  '_op.mat')
    f.save_data(stopInfoAll, srcDataDir + 'output/', target_file_name.replace('.mat','') + '_op_stop_info.mat', "stopInfo")

  0%|          | 0/423 [00:00<?, ?it/s]

Found 6 data files under data/v7_5_1_combined_preproc/


----------------------------------------------------------------------
Working on data/v7_5_1_combined_preproc\pp_cam1234567_lane1_combined_v7_5_1.mat...
Data position already in [m]]
(197659, 1)
data/v7_5_1_combined_preproc/pp_cam1234567_lane1_combined_v7_5_1.mat
There are 423 vehicles in this dataset.


100%|██████████| 423/423 [00:02<00:00, 192.44it/s]



runtime = 2.73


 30%|███       | 129/423 [00:00<00:00, 1289.98it/s]

...


runtime = 4.96


  3%|▎         | 15/583 [00:00<00:03, 149.79it/s]

Saved to data/v7_5_1_combined_preproc/output/pp_cam1234567_lane6_combined_v7_5_1.mat


100%|██████████| 583/583 [00:00<00:00, 698.08it/s]



runtime = 0.84
Saved to data/v7_5_1_combined_preproc/output/pp_cam1234567_lane6_combined_v7_5_1_op.mat
Saved to data/v7_5_1_combined_preproc/output/pp_cam1234567_lane6_combined_v7_5_1_op_stop_info.mat
